In [1]:
import pandas as pd
import time

In [2]:
def basic_set(df):
    basic = {}
    for i in df.drop_duplicates().values.tolist():
        basic[str(i)] = []
        for j, k in enumerate(df.values.tolist()):
            if k == i:
                basic[str(i)].append(j)

    return basic

def rough_set(data):
    data = data.dropna(axis=0, how='any')
    x_data = data.drop(['judge'], axis=1)
    y_data = data['judge']
    # 决策属性基本集
    y_basic_set = sorted([v for k, v in basic_set(y_data).items()])
    # 条件属性基本集
    x_basic_set = sorted([v for k, v in basic_set(x_data).items()])
    pos = []
    for i in x_basic_set:
        for j in y_basic_set:
            if set(i).issubset(j):
                pos.append(i)
    pos.sort()
#     print("x_basic_set",x_basic_set)
#     print("y_basic_set",y_basic_set)
#     print ('y的x正域Pos_x(y): ',pos)
    r_x_y = len([k for i in pos for k in i]) / (len(data))
    # print('依赖度r_x_(y):', r_x_y)

    # 探索条件属性中不可省关系
    columns_num = list(range(x_data.shape[1])) 
    
    # 收集核
    imp_core = []
    
    # 收集属性重要性
    imp_attr = []
    
    for i in columns_num:
        c = columns_num.copy()
        c.remove(i)
        u = data.iloc[:, c]
        u = sorted([v for k, v in basic_set(u).items()])
        pos_va = []
        
        for k in u:
            for j in y_basic_set:
                if set(k).issubset(j):
                    pos_va.append(k)
        
        if sorted(pos_va) != pos: 
            """
            如果去掉了了这个之后两个正域不相等，则是重要属性，放进imp_core
            """
            imp_core.append(i)
        
        r = len(sorted(pos_va)) / len(data)
        r_diff = round(r_x_y - r, 4)

        imp_attr.append(r_diff)

    dict_imp = {}
    for o, p in enumerate(imp_attr):
        dict_imp[data.columns[o]] = p

    result = dict_imp
    sorted_dict_imp = sorted(dict_imp, key=lambda x: dict_imp[x], reverse=True)
    sorted_dict_imp = list(map(lambda x: {x: dict_imp[x]}, sorted_dict_imp))
    imp_core = [data.columns[i] for i in imp_core]

    print('属性重要度(越大越重要)为:', sorted_dict_imp)
    print("imp_core:{}".format(imp_core))
    return dict_imp,sorted_dict_imp,imp_core

def deal(data):
    #获取数据长度
    len1 = data.shape[0]
    
    sorted_dict_imp = [[]]*len1
    total = [0]*len1
    title = data.columns.tolist()[:-1]
    dict_imp,sorted_dict_imp,imp_core = rough_set(data)
    return dict_imp,sorted_dict_imp,imp_core

In [3]:
time1 = time.time()
#读取文件数据
data = pd.read_csv(filepath_or_buffer='/Users/sijianxuan/Documents/test.csv')
dict_imp,sorted_dict_imp,imp_core = deal(data)
time2 = time.time()
print("runnning time is {}s".format(round(time2-time1,2)))

属性重要度(越大越重要)为: [{'C26': 0.0923}, {'C15': 0.0887}, {'C23': 0.0851}, {'C2': 0.0796}, {'C14': 0.0787}, {'C25': 0.0778}, {'C4': 0.0769}, {'C6': 0.0751}, {'C22': 0.0742}, {'C24': 0.0706}, {'C27': 0.0706}, {'C8': 0.0679}, {'C13': 0.0679}, {'C19': 0.067}, {'C1': 0.0661}, {'C10': 0.0652}, {'C18': 0.0652}, {'C3': 0.0643}, {'C12': 0.0643}, {'C16': 0.0643}, {'C17': 0.0643}, {'C5': 0.0633}, {'C7': 0.0633}, {'C21': 0.0633}, {'C9': 0.0624}, {'C11': 0.0624}, {'C20': 0.0624}, {'C28': 0.0615}]
imp_core:['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26', 'C27']
runnning time is 242.86s
